# Dokumentation (DE)

**Inhalt**
- 1. Einleitung & Ueberblick
- 2. GUI & Leveldesign
- 3. Spiellogik
- 4. Zusammenfassung & Ausblick


# 1. Einleitung & Ueberblick
- Spielidee: 2D-Plattformer (JavaFX). USB-Stick einsammeln, Dateien (Folder) sichern, Gefahren umgehen, Zielportal erreichen.
- Architektur: `Game` (Bootstrap, Config, Sound, Level), `ScreenManager` (Window, Input, Game Loop + delta), `GameScreen` (Rendering, Physik, Kamera, HUD, Zustandswechsel), `Level` + `TutorialLevel`/`SecondLevel`, `Platform` + `Block`-Ableitungen, `EntityPlayer`.
- Pipeline (Textdiagramm):
```text
Input -> InputManager -> GameScreen.update(delta)
  -> Physik (dx/dy, Gravitation, Kollision)
  -> Kamera (Dead-Zone + smoothing)
  -> Levelobjekte relativ zur Kamera
  -> HUD (Herzen, Files, Debug)
```


# 2. GUI & Leveldesign
## 2.1 Szene & Rendering
- `initialize()`: Root leeren, animierter Hintergrund, Spieler platzieren, Platforms/Blocks laden, HUD + Buttons + Pause-Overlay zeichnen.
- `update(delta)`: Physik, Input, Kollision, Kamera-Dead-Zone (`marginX/marginY`, `cameraSmooth`), HUD-Updates.
- `Platform.drawPlatform()`: Boden via Tileset `Material.FLOOR`, Kamera-Offset.
- `Block.update()`: Sprites an Kamera ausrichten.

## 2.2 HUD & Punkte
- Files/Score: `countFolderBlocks()` / `countActiveFolders()` -> Label `Files: X/Y`.
- Quest/Tooltips: `questLbl`, `tipsLbl` (F1), `flipperHint` nach Flipper.
- Health: `drawHealth()` mit Herz-Icons; Debug-Bar (F2); Buttons `Zurueck`/`Pause`.

## 2.3 Bewegte Objekte
- Spieler: dx/dy, Gravitation, Jump (`jumpPower`), Richtungs-Sprite (`Direction`), F3 NoClip/GodMode.
- Plattformen: `FloatingPlatformBlock` pendelt Start/Ende, `deltaX` zur Mitnahme; `EntityMovingPlatform` mit Hover/Sinus.
- Gegner/Projektil: `RobotEnemyBlock` patrouilliert und feuert `LaserBlock`; `BossRobotBlock` feuert nur, wenn Spieler oben. `LaserBlock` 4s Lebenszeit, 1 Schaden (kein Schaden im GodMode).
- Gefahren/Perks: `LavaBlock` tickt 0.25 HP; `GasBarrierBlock` deaktivierbar via E + Flipper (Fade); `JumpBoost`/`SpeedBoost` 10s Buff; `USBStick` schaltet Sammeln frei; `FolderBlock` Score nach Freigabe.

## 2.4 Spielzustaende
- Start: `Game.start()` -> Loading -> Menu -> GameScreen.
- Pause: `togglePause()` Overlay (Logik laeuft noch; TODO echter Halt).
- Game Over: HP <= 0 oder Sturz -> MainMenu.
- Finish: `FinishBlock.onCollide` -> `LevelFinishedScreen` -> Button ?Naechstes Level" setzt `SecondLevel` und neuen GameScreen.

## 2.5 Leveldesign
- Level 1: Boden + Lava-Graeben (`addLavaBetweenPlatforms`), USBStick, Jump/Speed-Boost, FloatingPlatformBlock, Finish, Tutorial-Flag in `onFinish()`.
- Level 2: dichtere Platforms, Lava-Saeulen, 8 FolderBlocks, JumpBoost-Route, FlipperItem + GasBarrier, BossRobot, Finish; einmaliger Spawn-Sound.
- Bedingung: Finish-Kollision; USB-Stick Pflicht fuer Files.


# 3. Spiellogik
## 3.1 Ziel & Ablauf
- Ziel: Sammeln freischalten (USB), Files einsammeln, Finish lebend erreichen.
- Startzustand: Health 5 Herzen; Start im Menu -> GameScreen.
- Enden: HP 0, Sturz, oder Finish-Kollision.

## 3.2 Steuerung & Interaktion
- Tasten: A/D laufen, SPACE springen (nur Boden), E interagieren (GasBarrier), W/S in NoClip, F1/F2/F3 Toggles.
- InputManager: `isPressed` Dauer, `pollJustPressed` entprellt.

## 3.3 Ereignisse & Logik
- Kollision: AABB, Reihenfolge Platforms dann Blocks; `pendingBlocks` fuer Laser.
- Schaden: Player klemmt HP, GodMode ignoriert; Lava/Gegner/Laser ziehen ab, Stomp killt Gegner.
- Timer/Delta: Bewegungen delta-basiert; Boost-Reset via `PauseTransition(10s)`; Laser 4s; `levelStarted` fuer Stats.
- Trennung: Logik in GameScreen/Blocks, Rendering via UIUtils; Kamera-Offsets in `update()`.

## 3.4 Klassen (Kurzbeschreibung)
- `Game`, `ScreenManager`, `GuiScreen`, `GameScreen`.
- `Level`, `TutorialLevel`, `SecondLevel`, `Platform`.
- `EntityPlayer`, `FloatingPlatformBlock`, `EntityMovingPlatform`.
- `RobotEnemyBlock`, `BossRobotBlock`, `LaserBlock`.
- `LavaBlock`, `GasBarrierBlock`, `JumpBoostBlock`, `SpeedBoostBlock`, `USBStickBlock`, `FolderBlock`, `FlipperItem/Block`, `FinishBlock`.


# 4. Zusammenfassung & Ausblick
- Kern stabil: GameScreen + ScreenManager + Level mit delta-Physik, Dead-Zone-Kamera, HUD/Debug.
- Anforderungen: Punkteanzeige (Files), Spielzustaende (HUD/Overlay), zwei Level, bewegte Objekte erfuellt.
- Ausblick: echte Pause-Logik, Level.update() nutzen, HUD um Timer/Score ergaenzen, Checkpoints, persistente Stats, Kamera-Clamps pro Level.
